In [1]:
using Markdown
using InteractiveUtils
using BenchmarkTools

# Reusing meaningful function names to define structural types
## Gustavo Nunes Goretkin
### JuliaCon 2021
- discuss functions used for their name and not for invocation
- identify pattern of setting up compuation, and performing it later
- experimental package (`FixArgs.jl`) to showcase this pattern
- focus on the idea rather than the specific implementation

Hi, my name is Gustavo Nunes Goretkin and I want to discuss this idea of using a function as a name, instead of invoking or calling it.

I am going to demonstrate this experimental package called FixArgs that showcases some of these patterns.

The point of the talk is to focus on the idea rather than the specific implementation in FixArgs.

To concatenate two `AbstractVector`s, use `vcat`:

To get started, let's use an example based on vcat, which is how we concatenate vectors.

To concatenate a vector of vectors, we can use the higher-order function `reduce` with `vcat`.

In [ ]:
vcat(1:5, 1:5) |> string

Given a `Vector` of `Vector`s,

In [3]:
vs = [collect(1:10) for _ = 1:2000]; # `Vector` of `<:AbstractVector`s,  really.

we can concatenate all of the inner `Vector`s with

In [4]:
reduce(vcat, vs) |> string

"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1, 2, 

Now let us do essentially the same computation, but instead of directly using `vcat`, we define a function that just calls `vcat`:

```julia
reduce((_1, _2) -> vcat(_1, _2), vs)
```

And let's time the two.

Let's repeat that same computation, but instead of using `vcat` directly, we'll use it via an anonymous function.

In [5]:
@benchmark reduce(vcat, vs)

BenchmarkTools.Trial: 
  memory estimate:  172.08 KiB
  allocs estimate:  3
  --------------
  minimum time:     20.859 μs (0.00% GC)
  median time:      82.838 μs (0.00% GC)
  mean time:        101.635 μs (13.48% GC)
  maximum time:     5.010 ms (97.35% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [6]:
@benchmark reduce((_1, _2) -> vcat(_1, _2), vs)

BenchmarkTools.Trial: 
  memory estimate:  76.74 MiB
  allocs estimate:  3592
  --------------
  minimum time:     35.989 ms (11.26% GC)
  median time:      41.300 ms (21.73% GC)
  mean time:        41.066 ms (19.50% GC)
  maximum time:     47.448 ms (22.00% GC)
  --------------
  samples:          122
  evals/sample:     1

It is ~500× slower in this case.

Why? It is not because anonymous functions are slow (they aren't). Totally different code is running each case. (e.g. allocate output all at once)

In this case, overall the first call is 500 times faster. Why?
It's because there's totally different code running in each case.

If we look at the methods of reduce, we see that there is a special case.


`methods(reduce)` shows
```julia
[4] reduce(::typeof(vcat), A::AbstractVector{var"#s79"} where var"#s79"<:(AbstractVecOrMat{T} where T)) in Base at abstractarray.jl:1569
```

## Spelling
There is a reason that "spelling bees" were invented for English-language writing system.

Two key features:
1. multiple dispatch
2. the function is encoded in the type domain (i.e. each function is its own type)

enables defining special case for `reduce(vcat, ...)`.

Otherwise, would need name e.g. `reduce_vcat(...)`, or perhaps `flatten(...)`.

Personally, the first spelling is better. It combines existing and meaningful names instead of introducing a new ad-hoc name.

There are two key features that together enable the reduce vcat special case.

One is that we have multiple dispatch, which allows the method to exist in the first place.

And the second is that functions are encoded in the type domain. Each function is its own type.

Without this, we might need reduce underscore vcat, or a more familiar name such as flatten.

I think the first spelling is better because it combines existing names reduce and vcat in a meaningful way, instead of creating an ad-hoc name.

Note that the method might not call `vcat`. `vcat` is used as a name.

And this matters because, more so than in other ecosystems, the Julia community aims to determine a generic meaning for function names.

This is difficult and fundamental design work, and the care taken here is what makes generic programming as successful as it is in Julia.

Naming things is hard, and it would be wise to reuse these names as much as possible.

Calling `reduce(vcat, ...)` might not call `vcat`. `vcat` is used as a _name_.

More so than in other ecosystems, the Julia community aims to determine a generic meaning for a given function name. This is difficult and fundamental design work. It enables generic programming. It is good to reuse these names!

There are multiple converging motivations for the idea in this talk:

* Get extra value from careful function name / meaning pairs

* Generalize `Base.Fix1`/`Base.Fix2`

* Symbolic Computation / Lazy Computation

* Explore a combination of structural and of nominal types

## Demo of `Base.Fix2`
The original motivation for `FixArgs.jl` is to generalize `Base.Fix1` and `Base.Fix2`.

Let's talk about `Base.Fix2`, using `==`, this two-argument function.

So, we're going to define `f1` and `f2`. For the same input, they give the same output, which is whether the argument equals 50.

Notice that the type of `f1` is this `Fix2` type, which itself mentions `==` and the type of 50.
`f2` just has this name, which I guess is the 7th anonymous function defined.

We want `f1` and `f2` to behave identically, and when passed to `findfirst`, they give the same result. But I can tell you that this first use is a constant-time operation, and this second use is linear in the number of elements in this range.

`==` is a two-argument function. `==(a, b)` is identical to `a == b`

In [7]:
f1 = ==(50)

(::Base.Fix2{typeof(==), Int64}) (generic function with 1 method)

In [8]:
f2 = x -> x == 50

#7 (generic function with 1 method)

`f1` and `f2` compute the same function. Some might say that they are different names for the same function.



And in this case, names matter!

In [9]:
findfirst(f1, -1000:1000) # O(1)

1051

In [10]:
findfirst(f2, -1000:1000) # O(length(range))

1051

In [11]:
@which findfirst(f1, -1000:1000)

findfirst(p::Union{Base.Fix2{typeof(==), T}, Base.Fix2{typeof(isequal), T}}, r::AbstractUnitRange) where T<:Integer in Base at array.jl:1912

And the reason this happens is because there is a `findfirst` method special case. It's definition is what you expect.

This is one thing I love about Julia. You can imagine a plotting library that supports axis ticks. You can use an object like this to represent evenly-spaced ticks, or use a `Vector` to represent arbitrary ticks.

Code written in terms of `findfirst` and other functions like it can support both types of ticks, and it will be efficient for the evenly-spaced ticks and still work for the general case.


```julia
findfirst(p::Union{Fix2{typeof(isequal),T},Fix2{typeof(==),T}}, r::AbstractUnitRange) where {T<:Integer} =
    first(r) <= p.x <= last(r) ? 1+Int(p.x - first(r)) : nothing
```

This is why I love Julia. Imagine a plotting library that supports unevenly-spaced axis ticks.

Code written in terms of `findfirst` can support unevenly-spaced ticks, and still be (runtime) efficient when using evenly-spaced ticks.

The same code can handle a general case, without affecting performance in the special case!

The types `Fix1`/`Fix2` fix (or bind, or partially apply) one argument of a two-argument function.


`FixArgs.jl` provides a generalization of `Fix1` and `Fix2` in a few ways:
1. A function of any positional arity can be used, and any number of its arguments can be bound, allowing the remaining arguments to be provided later.
2. A function can have its keyword arguments bound.
3. The function `x -> f(x, b)` is represented with types:
   - a [`Lambda`](https://goretkin.github.io/FixArgs.jl/dev/#FixArgs.Lambda) to represent function (`args -> body`)
   - a [`Call`](https://goretkin.github.io/FixArgs.jl/dev/#FixArgs.Call) to represent the function *call* (`f(...)`) in the body
   - a [`ArgPos`](https://goretkin.github.io/FixArgs.jl/dev/#FixArgs.ArgPos) to represent the `x` in the body of the lambda function

So Fix1 and Fix2 fix respectively the first or second argument of a two-argument function.
The package we're discussing here provides a generalization in a few ways.

The function can have any number of arguments and any pattern of them can be fixed.

The keyword arguments can be bound

We can represent call expressions on their own, not just partially-applied functions.

## A quick fix

Partial application with keyword arguments

```julia
"""
    isapprox(x; kwargs...) / ≈(x; kwargs...)
Create a function that compares its argument to `x` using `≈`, i.e. a function equivalent to `y -> y ≈ x`.
The keyword arguments supported here are the same as those in the 2-argument `isapprox`.
"""
isapprox(y; kwargs...) = x -> isapprox(x, y; kwargs...)
```

All other docstrings in `Base` with "Create a function" use `Base.Fix2`, but here an anonymous function is used.

So a quick example of where fixing keyword arguments is useful is in `isapprox`. All other partially-applied functions in `Base`. use `Fix2`, but this one has keyword arguments and so it just gets this lambda function.

## Illustration
Do we want a way to encode lambda calculus in types?

Would it ever be useful to fix all of the arguments of a function?

So, do we want to encode lambda calculus in types in Julia? Is it ever useful to fix all of the arguments of a function?

The answer to me is definitely yes!

Let's consider the division function. If you fix its two arguments, to me it looks a lot like a rational number.

Said differently, a rational number is kind of like lazy division.

So here we'll start using the package actually.

This is how we could encode the rational 1 over 2.
...

If we multiply two of these objects together, we get a method error.

Consider the `/` (division) function.

If you fix its two arguments, that looks a lot like `Rational`.

In [12]:
using FixArgs
half = @xquote 1 / 2

Call(Some(/), FrankenTuple((Some(1), Some(2)), NamedTuple()))

A macro helps create `Lambda` / `Call` / `ArgPos`, etc. objects

In [13]:
half * half

LoadError: MethodError: no method matching *(::FixArgs.Call{Some{typeof(/)}, FrankenTuples.FrankenTuple{Tuple{Some{Int64}, Some{Int64}}, (), Tuple{}}}, ::FixArgs.Call{Some{typeof(/)}, FrankenTuples.FrankenTuple{Tuple{Some{Int64}, Some{Int64}}, (), Tuple{}}})
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560

Another macro helps define the types

We can define a method using some helpful macros.

Here is an example definition of how to multiply an S divided by S by an S divided by S.

Now when we do a half times half we get a representation of a quarter.

Note that the dollar sign is used to force evaluation, without it we would get.


In [14]:
function Base.:*(a::(@xquoteT ::T / ::T), b::(@xquoteT ::T / ::T)) where T
    (n1, d1) = something.(Tuple(a.args))
    (n2, d2) = something.(Tuple(b.args))
    @xquote $(n1 * n2) / $(d1 * d2)
end

In [15]:
half * half

Call(Some(/), FrankenTuple((Some(1), Some(4)), NamedTuple()))

Note that `$` is used to escape "xquote"ing.

In [16]:
let n1 = n2 = 1, d1 = d2 = 2
    @xquote (n1 * n2) / (d1 * d2)
end

Call(Some(/), FrankenTuple((Call(Some(*), FrankenTuple((Some(1), Some(1)), NamedTuple())), Call(Some(*), FrankenTuple((Some(2), Some(2)), NamedTuple()))), NamedTuple()))

Where does the connection between `Rational` and `/` happen in `Base`? [Here](https://github.com/JuliaLang/julia/blob/51f57406038df9f438edf911c8d1b59c9c1af4b1/base/rational.jl#L112).

    
```julia
AbstractFloat(x::Rational) = (float(x.num)/float(x.den))::AbstractFloat
```

In Base, the connection between the `Rational` type and division happens in this method.

In FixArgs, you can use this `xeval` function which evaluates the form. So the type itself already encodes that it is the division of two things.

In [17]:
xeval(half) # evaluate the lazy representation

0.5

## memory and code generation comparison

So I want to compare some low-level details.

First of all, the size of this object is the same as the corresponding `Rational{Int}`.

Even the memory layout is identical.

In [18]:
sizeof(half)

16

In [19]:
sizeof(1 // 2)

16

In [20]:
reinterpret(Int, [half])

2-element reinterpret(Int64, ::Vector{FixArgs.Call{Some{typeof(/)}, FrankenTuples.FrankenTuple{Tuple{Some{Int64}, Some{Int64}}, (), Tuple{}}}}):
 1
 2

In [21]:
@code_native xeval(half)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ eval.jl:79 within `xeval' @ eval.jl:44
; │┌ @ eval.jl:104 within `_xapply'
; ││┌ @ FrankenTuples.jl:405 within `ftcall'
; │││┌ @ int.jl:93 within `/'
; ││││┌ @ float.jl:206 within `float'
; │││││┌ @ float.jl:191 within `AbstractFloat'
; ││││││┌ @ float.jl:94 within `Float64'
	vcvtsi2sdq	(%rdi), %xmm0, %xmm0
	vcvtsi2sdq	8(%rdi), %xmm1, %xmm1
; ││││└└└
; ││││ @ int.jl:93 within `/' @ float.jl:335
	vdivsd	%xmm1, %xmm0, %xmm0
; │└└└
; │ @ eval.jl:79 within `xeval'
	retq
; └


In [22]:
@code_native float(1//2)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ float.jl:206 within `float'
; │┌ @ rational.jl:112 within `AbstractFloat'
; ││┌ @ float.jl:206 within `float'
; │││┌ @ float.jl:191 within `AbstractFloat'
; ││││┌ @ float.jl:94 within `Float64'
	vcvtsi2sdq	(%rdi), %xmm0, %xmm0
	vcvtsi2sdq	8(%rdi), %xmm1, %xmm1
; ││└└└
; ││┌ @ float.jl:335 within `/'
	vdivsd	%xmm1, %xmm0, %xmm0
; │└└
	retq
; └


Let's look at the code that's generated to convert this rational to a float. It's also identical.

This is not a drop-in replacement for `Rational`, however, because

In [23]:
1//2 isa Number

true

In [24]:
(@xquote 1 / 2) isa Number

false

That said, it's not a drop-in replacement for Rational because `Rational` is a subtype of `Number`, and this `FixArgs.Call` object is not.

## Why?
Using this instead of `Base.Rational` seems pretty silly and confusing. Possible benefits:

* Some users want a "rational" type where the numerator and the denominator are not constrained to be the same type.

* a fixed-point number is one of these rational types where the denominator is "static" (a singleton type such that the numerical value is encoded in the type domain). Check the documentation for `FixArgs.jl` examples!

In [25]:
MyQ0f7(x) = (@xquote $(Int8(x)) / 128::::S) # mark an argument as "static"
MyQ0f7(3)

Call(Some(/), FrankenTuple((Some(3), Val{128}()), NamedTuple()))

So replacing Base.Rational with this is probably silly and confusing, but there are some possible benefits.

Some users want to have different types for the numerator and the denominator.

A perfect example of this is fixed point numbers, where the denominator is a "static" number encoded in the type.

There are more examples in the documentation, but I'll give you a quick look that we can use this quadruple colon S to mark this as static, and then the value here is in the type domain.

### A combination of structural and nominal types
Avoid the requirement to choose names like `Rational`, `num`, `den`.
The arguments can be distinguished by the role they play with respect to the function. Only the function needs a name.

("structural" and "nominal" are used to describe type *systems*, not *types*, and I am co-opting the term here)

In [26]:
@xquote x -> x # `identity` could be an alias for this

Lambda(FixArgs.Arity{1, Nothing}(), arg_pos(1, 0))

In [27]:
x -> x

#9 (generic function with 1 method)

In [28]:
x -> x # gets a different name, even though structurally identical.

#11 (generic function with 1 method)

In [29]:
@xquote x -> x((y -> x))

FixNew(FixArgs.Arity{1, Nothing}(),arg_pos(1, 0),FrankenTuple((Lambda(FixArgs.Arity{1, Nothing}(), arg_pos(1, 1)),), NamedTuple()))

There is a kind of combination of structural and nominal typing here, because you can avoid choosing names like `Rational`, `num` and `den`. The arguments can be distinguished by the role they play with respect to the function. Only the function needs a name, and the rest is determined structurally.

The same point can be made differently. So if we xquote an identity function, we get this value every single time.

That's different from defining anonymous functions in Julia because these two do not compare the same, even though they are structurally identical.

And in case you're wondering, FixArgs does support all sorts of nested lambda expressions correctly, as you'd hope.

## Existing patterns
Any "eager" function now has a lazy representation.

So what are some existing patterns that this relates to? Now, any eager function has a lazy representation.

`Base.Generator` could be a lazy representation of `map(f, itr)`

```julia
Base.Iterators.Filter(flt, itr)
```

could be replaced with

```julia
@xquote filter(flt, itr)
```

The type `Base.Iterators.Filter` could be an alias for

```julia
(@xquoteT filter(::F, ::I)) where {F, I}
```
to keep the existing symbolic "rules" such as [this one](https://github.com/JuliaLang/julia/blob/ef14131db321f8f5a815dd05a5385b5b27d87d8f/base/iterators.jl#L463):

```julia
reverse(f::Filter) = Filter(f.flt, reverse(f.itr))
```

[`Base.Iterators.Flatten(iterator_of_iterators)`](https://github.com/JuliaLang/julia/blob/ef14131db321f8f5a815dd05a5385b5b27d87d8f/base/iterators.jl#L1038-L1040) can be represented as `@xquote reduce(vcat, iterator_of_iterators)`

### `Base.literal_pow` 

In [30]:
Meta.@lower x^2

:($(Expr(:thunk, CodeInfo(
    @ none within `top-level scope'
1 ─ %1 = Core.apply_type(Base.Val, 2)
│   %2 = (%1)()
│   %3 = Base.literal_pow(^, x, %2)
└──      return %3
))))

Could instead lower to approximately
```
%1 = @xquote x^(2::::S) # i.e. something roughly like `Call(^, x, Val(2))`
%2 = xeval(%1)
```

### Broadcasting
[Call](https://github.com/JuliaLang/julia/blob/d06c2a97be3f643d403c4069955e135823ff9fd0/base/broadcast.jl#L152-L173)

```julia
struct Broadcasted{Style<:Union{Nothing,BroadcastStyle}, Axes, F, Args<:Tuple} <: Base.AbstractBroadcasted
    f::F
    args::Args
    axes::Axes
end
```

(extra information `Style` and `axes`)

[xeval](https://github.com/JuliaLang/julia/blob/1910a7685a86d44041a98ff874f92e480fc44632/base/broadcast.jl#L904)

```julia
@inline materialize(bc::Broadcasted) = copy(instantiate(bc))
# [... `copyto!` methods ...]
```

[LazyArrays.jl](https://github.com/JuliaArrays/LazyArrays.jl)

[Call](https://github.com/JuliaArrays/LazyArrays.jl/blob/6b5cee2e8ed355a8e7590c3ae860faed4e816e19/src/lazyapplying.jl#L184-L187)

```julia
struct ApplyArray{T, N, F, Args<:Tuple} <: LazyArray{T,N}
    f::F
    args::Args
end

# [...]

const Vcat{T,N,I<:Tuple} = ApplyArray{T,N,typeof(vcat),I}

```

[xeval](https://github.com/JuliaArrays/LazyArrays.jl/blob/dff5924cd8b52c62a34cce16372381bb8a9e35cb/src/lazyconcat.jl#L20-L27)
```julia
function instantiate(A::Applied{DefaultApplyStyle,typeof(vcat)})
    isempty(A.args) && return A
    m = size(A.args[1],2)
    for k=2:length(A.args)
        size(A.args[k],2) == m || throw(ArgumentError("number of columns of each array must match (got $(map(x->size(x,2), A)))"))
    end
    Applied{DefaultApplyStyle}(A.f,map(instantiate,A.args))
end
```


[LazyStacks.jl](https://github.com/mcabbott/LazyStack.jl)

[Call](https://github.com/mcabbott/LazyStack.jl/blob/a02740adae65cd27acc0257a61a482fdebf2e2eb/src/LazyStack.jl#L82-L84)
```julia
struct Stacked{T,N,AT} <: AbstractArray{T,N}
    slices::AT
end
```


[xeval](https://github.com/mcabbott/LazyStack.jl/blob/a02740adae65cd27acc0257a61a482fdebf2e2eb/src/LazyStack.jl#L111)
```julia
Base.collect(x::Stacked{T,2,<:AbstractArray{<:AbstractArray{T,1}}}) where {T} = reduce(hcat, x.slices)
```

[LazySets.jl](https://github.com/JuliaReach/LazySets.jl)

[Call](https://github.com/JuliaReach/LazySets.jl/blob/4e3dc1668a0265f748fef4a5ced93c97337c2623/src/LazyOperations/Intersection.jl#L98-L108)

```julia
struct Intersection{N, S1<:LazySet{N}, S2<:LazySet{N}} <: LazySet{N}
    X::S1
    Y::S2
    cache::IntersectionCache
    # [...]
end
```

[xeval](https://github.com/JuliaReach/LazySets.jl/blob/4e3dc1668a0265f748fef4a5ced93c97337c2623/src/LazyOperations/Intersection.jl#L723-L725)
```julia
function concretize(cap::Intersection)
    return intersection(concretize(cap.X), concretize(cap.Y))
end
```

### Struct-of-Arrays and Array-of-Structs

In [31]:
soa = (a=[1,2,3], b=[10, 20, 30])
aos = [(a=1, b=10), (a=2, b=20), (a=3, b=30)]

aos_eager = map(NamedTuple{(:a, :b)} ∘ tuple, soa.a, soa.b)

3-element Vector{NamedTuple{(:a, :b), Tuple{Int64, Int64}}}:
 (a = 1, b = 10)
 (a = 2, b = 20)
 (a = 3, b = 30)

In [32]:
soa_eager = NamedTuple{(:a, :b)}(tuple(getindex.(aos, :a), getindex.(aos, :b)))

(a = [1, 2, 3], b = [10, 20, 30])

In [33]:
@xquote map(NamedTuple{(:a, :b)} ∘ tuple, soa.a, soa.b)

Call(Some(map), FrankenTuple((Call(Some(∘), FrankenTuple((Some(NamedTuple{(:a, :b), T} where T<:Tuple), Some(tuple)), NamedTuple())), Some([1, 2, 3]), Some([10, 20, 30])), NamedTuple()))

September 2020:
The composition operator `∘` now returns a [`Base.ComposedFunction`](https://github.com/jw3126/julia/blob/37908d66492f3cf95759f8e0e7e3d2bd6d038c0c/base/operators.jl#L909-L914) instead of an anonymous function ([#37517]).

```julia
struct ComposedFunction{O,I} <: Function
    outer::O
    inner::I
end
```

## Different from `Expr`
`/` in `:(/(x, y))` is not `Base.:/`, but just `:/`

In [34]:
dump(:(x / y))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol /
    2: Symbol x
    3: Symbol y


In [35]:
let x = 1, y =2
    @xquote x / y
end

Call(Some(/), FrankenTuple((Some(1), Some(2)), NamedTuple()))

Note that this is different than using the expression type in Base.

When we quote x divided by y, we get a bunch of `Symbol`s, but when using xquote, we actually evaluate the x, the y, and the function.

## Not for Computer Algebra System

[SymbolicUtils.Term](https://github.com/JuliaSymbolics/SymbolicUtils.jl/blob/c3081cdbf59b8e07ed3a757d8e9eb8bdbc9cad6e/src/types.jl#L323-L328) intentionally does not dispatch on function or argument types:

```julia
struct Term{T, M} <: Symbolic{T}
    f::Any
    arguments::Any
    metadata::M
    hash::Ref{UInt} # hash cache
end
```


In this `Term` type from `SymbolicUtils`, the function and the arguments are of type `Any`, and do not show up in the type. This is intentional

## Taking the idea too far?

In [36]:
typeof(im)

Complex{Bool}

`im` is defined in terms of `Complex`. Going the other way

In [37]:
𝗶 = @xquote sqrt((-1)::::S)

Call(Some(sqrt), FrankenTuple((Val{-1}(),), NamedTuple()))

In [38]:
xeval(𝗶)

LoadError: DomainError with -1.0:
sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

In [39]:
c = @xquote 1 + 2*𝗶

Call(Some(+), FrankenTuple((Some(1), Call(Some(*), FrankenTuple((Some(2), Some(Call(Some(sqrt), FrankenTuple((Val{-1}(),), NamedTuple())))), NamedTuple()))), NamedTuple()))

In [40]:
reinterpret(Int, [c])

2-element reinterpret(Int64, ::Vector{FixArgs.Call{Some{typeof(+)}, FrankenTuples.FrankenTuple{Tuple{Some{Int64}, FixArgs.Call{Some{typeof(*)}, FrankenTuples.FrankenTuple{Tuple{Some{Int64}, Some{FixArgs.Call{Some{typeof(sqrt)}, FrankenTuples.FrankenTuple{Tuple{Val{-1}}, (), Tuple{}}}}}, (), Tuple{}}}}, (), Tuple{}}}}):
 1
 2

Note that
- `@xquote 1 + 2*𝗶`
- `@xquote 2*𝗶 + 1`
- `@xquote 𝗶*2 + 1`
are all different types.

(https://en.wikipedia.org/wiki/Deductive_lambda_calculus#Intensional_versus_extensional_equality)

## Downsides
- code reuse increases coupling
- large types and compile times
- method ambiguities

# Thanks!